# RAG_Context_Relevancy_Checker


[RAG_Context_Relevancy_Checker](https://medium.com/the-ai-forum/rag-context-relevancy-checker-agent-using-deepseek-r1-70b-on-groq-modernbert-langchain-58edb6b0f29c)


## SETUP
